In [1]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer  # binaryclass
import cv2 as cv
import os
from Layer import Layer
from NN import NN
import numpy as np
from sys import path
path.append('..')
import imutils

In [3]:
### Import data

path_human = 'Human-2'
path_non_human = 'Non-Human-2'
ones_human = np.ones((64*64, 1)).reshape(1, -1)

# convert image shape to 1x4096 and put into a matrix ones_human

for file in os.listdir(path_human):
    path_img_human = path_human+'/'+file
    image_human = cv.imread(path_img_human, 0).reshape(1, -1)
    ones_human = np.vstack((ones_human, image_human))

data_human = np.delete(ones_human, 0, 0)

# N = number of data set, d is the number of features

N, d = data_human.shape
# print(N, d)


ones_non_human = np.ones((64*64, 1)).reshape(1, -1)
for file in os.listdir(path_non_human):
    path_img_non_human = path_non_human+'/'+file
    image_non_human = cv.imread(path_img_non_human, 0).reshape(1, -1)
    ones_non_human = np.vstack((ones_non_human, image_non_human))

data_non_human = np.delete(ones_non_human, 0, 0)

feature_set = np.vstack((data_human, data_non_human))

label_ones = np.ones((data_human.shape[0], 1))
label_zeros = np.zeros((data_non_human.shape[0], 1))
targets = np.vstack((label_ones, label_zeros))

X_train, X_test, Y_train, Y_test = train_test_split(
    feature_set, targets, test_size=0.2)


In [1]:
### train the network
### skip this step if already have the pre-trained weights

# create the network
nn_model = NN(X_train, Y_train)
nn_model.add_layer(Layer(50, activation='relu'))
nn_model.add_layer(Layer(25, activation='sigmoid'))


# fit the network
nn_model.fit(iteration=20000, learning_rate=0.001)

# plot cost function

Y_train_pred = nn_model.predict(X_train)
Y_test_pred = nn_model.predict(X_test)

accuracy_train = np.sum(Y_train_pred == Y_train) / len(Y_train) * 100
accuracy = np.sum(Y_test_pred == Y_test) / len(Y_test) * 100
print("Train accuracy: " + str(accuracy_train) + "%")
print("Test accuracy: " + str(accuracy) + "%")

NameError: name 'NN' is not defined

In [4]:
### predict with pretrained weights, still need to import data

# create the network
nn_model_2 = NN(X_train, Y_train)
nn_model_2.add_layer(Layer(50, activation='relu'))
nn_model_2.add_layer(Layer(25, activation='sigmoid'))

# plot cost function
# Y_train_pred_2 = nn_model_2.predict_pretrained_weights(X_train)
Y_test_pred_2 = nn_model_2.predict_pretrained_weights(X_test)

# accuracy_2_train = np.sum(Y_train_pred_2 == Y_train) / len(Y_train) * 100
# print("Train accuracy 2: " + str(accuracy_2_train) + "%")
accuracy_2 = np.sum(Y_test_pred_2 == Y_test) / len(Y_test) * 100
print("Test accuracy 2: " + str(accuracy_2) + "%")

Test accuracy 2: 98.64430147058823%


In [5]:
### link
# https://stackoverflow.com/questions/37177811/crop-rectangle-returned-by-minarearect-opencv-python

def crop_minAreaRect(rect, box, gray, frame):
    mult = 1 # I wanted to show an area slightly larger than my min rectangle set this to one if you don't
    W = rect[1][0]
    H = rect[1][1]

    Xs = [i[0] for i in box]
    Ys = [i[1] for i in box]
    x1 = min(Xs)
    x2 = max(Xs)
    y1 = min(Ys)
    y2 = max(Ys)

    rotated = False
    angle = rect[2]

    if angle < -45:
        angle+=90
        rotated = True

    center = (int((x1+x2)/2), int((y1+y2)/2))
    size = (int(mult*(x2-x1)),int(mult*(y2-y1)))

    M = cv.getRotationMatrix2D((size[0]/2, size[1]/2), angle, 1.0)

    cropped = cv.getRectSubPix(gray, size, center)    
    cropped = cv.warpAffine(cropped, M, size)

    croppedW = W if not rotated else H 
    croppedH = H if not rotated else W

    croppedRotated = cv.getRectSubPix(cropped, (int(croppedW*mult), int(croppedH*mult)), (size[0]/2, size[1]/2))
#     cv.circle(frame, center, 10, (0,255,0), -1) # again this was mostly for debugging purposes
#     cv.rectangle(frame, 
#                  (int(center[0] - croppedW/2), int(center[1] - croppedH/2)), 
#                  (int(center[0] + croppedW/2), int(center[1] + croppedH/2)), 
#                  (255, 255, 255), 
#                  2) # again this was mostly for debugging purposes
    
    return croppedRotated

In [ ]:
### test with video

backSub = cv.BackgroundSubtractorMOG2(detectShadows=False)
capture = cv.VideoCapture('12.mp4')
fps = 30
MIN_AREA = 600

if not capture.isOpened:
    print('Unable to open the video')
    exit(0)

while True:
    ret, frame = capture.read()
    if frame is None:
        break

    # convert to gray + resize + apply filter
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    gray_copy = gray.copy()
    gray = cv.GaussianBlur(gray, (21, 21), 0)

    fgMask = backSub.apply(gray)
    fgMask = cv.medianBlur(fgMask,5)
    
    # get contours
    cnts = cv.findContours(
        fgMask.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    for c in cnts:
        rect = cv.minAreaRect(c)
        if (rect[1][0]*rect[1][1]) < MIN_AREA:
            continue
        
        box = cv.boxPoints(rect)
        box = np.int0(box)
        cv.drawContours(frame,[box],0,(0,255,0),2)
        
        img_crop = crop_minAreaRect(rect, box, gray_copy, frame)       
        img_crop = cv.resize(img_crop, (64,64), interpolation = cv.INTER_AREA)
        
        Y_box_pred = np.sum(nn_model_2.predict(img_crop.reshape(1, -1)))
        
        if Y_box_pred == 1:
            cv.putText(frame, "Person", (box[2][0], box[2][1]),
               cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        if Y_box_pred == 0:
            cv.putText(frame, "Non", (box[2][0], box[2][1]),
               cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
        
        
        
        
        
#         if cv.contourArea(c) < MIN_AREA:
#             continue

#         (x, y, w, h) = cv.boundingRect(c)
#         cut_gray = gray_copy[y:y+h,x:x+w]
#         cut_gray = cv.resize(cut_gray, (64,64), interpolation = cv.INTER_AREA)
#         Y_box_pred = np.sum(nn_model_2.predict(cut_gray.reshape(1, -1)))
#         if Y_box_pred == 1:
#             cv.putText(frame, "Person", (x, y-10),
#                cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
#         if Y_box_pred == 0:
#             cv.putText(frame, "Non", (x, y-10),
#                cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
#         cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  
        
    cv.imshow('Frame', frame)
#     cv.imshow('FG Mask', fgMask)
#     cv.imshow('Gray copy', gray_copy)
    if cv.waitKey(int((1/fps)*1000)) & 0xFF == ord('q'):
        break

capture.release()
cv.destroyAllWindows()